# Import Libraries and Set Random Seeds

This section imports all packages required for the context of this project

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
import pickle
import random as python_random
import os

Imports for Model Architecture

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50, InceptionResNetV2, VGG16, InceptionV3, DenseNet201
from keras import models, layers
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint

Imports for Model Evaluation

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, f1_score
from sklearn.model_selection import train_test_split

Set random seeds for reproducible dataset

In [ ]:
np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

Additional checks done to ensure required folders are already created, if not create it. The following folders will be checked.

1. `./weights`: Store the optimal weights of every model trained, named as `<model_name>.h5`
2. `./results`: Store the visualisation of each models performance
    * `./results/acc_loss_plots`: Store the loss and accuracy curves of each model's training
    * `./results/metrics`: Store the f1 and auc score comparison bar graphs
    * `./results/cm`: Store the confusion matrix generated by each model upon the validation set

In [85]:
if not os.path.exists('./weights'):
    os.mkdir('./weights')

if not os.path.exists('./results'):
    os.mkdir('./results')

subfolders = ['acc_loss_plots', 'clfr', 'cm', 'metrics']
for sf in subfolders:
    filepath = './results/{}'.format(sf)
    if not os.path.exists(filepath):
        os.mkdir(filepath)

# Download and Preprocess Dataset

The dataset used, the IMDB-WIKI dataset faces can be downloaded using the following terminal command and preprocessed using the `mat.py` file which was obtained from the following repository: https://github.com/imdeepmind/processed-imdb-wiki-dataset/blob/master/mat.py

In [ ]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar

In [ ]:
%run mat.py

# Load Dataset

This section loads the dataset, splits the dataset into Training, Testing and Validation and loads these split datasets into separate `ImageDataGenerator`.

Additionally, existing results from previous sessions are loaded and saved for ease of collaboration and evaluation in the future. 

  Model histories are saved into a combined `histories.pickle` file as a dictionary with the following structure:
{
    `model_name`: Keras `history` callback object
}. 

  The maximum validation accuracy, F1 score and AUC_ROC score are saved into the `model_performance.csv` file in the following format: <`model_name`, `max_val_acc`, `f1_score`, `roc_auc`>.

In [ ]:
df = pd.read_csv('meta.csv')
df

The class distribution is explored to examine if any additional steps is required to ensure a balanced class distribution

In [ ]:
sns.countplot(x=df['gender'])
plt.show()

We see that this is an imbalanced dataset and hence requires a stratified split into training, testing and validation.

In [ ]:
# First take out 20% for test (this dataset will not be used in the training process)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=0, stratify=df['gender'])
# Then split the remaining 80% into training and validation
train_df, val_df = train_test_split(train_df, test_size=0.3, random_state=0, stratify=train_df['gender'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

Training, Validation and Testing geneerators are defined. In addition, a `predict_generator` is added on which model evaluation of generating the F1 and ROC_AUC scores is performed. This additional generator is required so that a comparison can be made by setting the parameter `shuffle=False`.

In [ ]:
def generate_data(img_size=224, batch_size=128):
  # data generator for training data
  train_generator = train_datagen.flow_from_dataframe(train_df, x_col="path", 
                                                      y_col="gender", 
                                                      target_size=(img_size,img_size), 
                                                      batch_size=batch_size, 
                                                      class_mode='binary')

  # data generator for validation data
  validation_generator = validation_datagen.flow_from_dataframe(val_df, x_col="path", 
                                                                y_col="gender", 
                                                                target_size=(img_size,img_size),
                                                                batch_size=batch_size, 
                                                                class_mode='binary')

  # data generator for validation data to be used for prediction
  predict_generator = validation_datagen.flow_from_dataframe(val_df, x_col="path", 
                                                             y_col="gender", 
                                                             target_size=(img_size,img_size),
                                                             batch_size=batch_size, 
                                                             class_mode='binary',
                                                             shuffle = False)

  # data generator for testing data
  test_generator = test_datagen.flow_from_dataframe(test_df, x_col="path", 
                                                    y_col="gender", 
                                                    target_size=(img_size,img_size),
                                                    batch_size=batch_size, 
                                                    class_mode='binary',
                                                    shuffle = False)
  
  generators = {'train_gen': train_generator,
                'validation_gen': validation_generator,
                'test_gen': test_generator,
                'predict_gen': predict_generator}
  
  return generators

Generate the most updated `histories` object and explore which models have been fit.

In [ ]:
def save_history():
    hist_out_file = open("histories.pickle", "wb")
    pickle.dump(histories, hist_out_file)
    hist_out_file.close()

def open_history():
    try:
        file = open("histories.pickle", 'rb')
        pickleData = pickle.load(file)
        file.close()
    except (OSError, IOError) as e:
        pickleData = {}
        pickle.dump(pickleData, open("histories.pickle", "wb"))
    return pickleData

In [ ]:
histories = open_history()
histories.keys()

Generate the most updated `performance_df` dataframe.

In [ ]:
def open_performance():
    if os.path.exists('model_performance.csv'):
        return pd.read_csv("model_performance.csv", index_col=False)
    else:
        return pd.DataFrame(columns=['model_name', 'max_val_acc', 'error', 'f1_score', 'roc_auc'])
        
def save_performance(model_name, error, f1, roc_auc):
    global performance_df
    new_row = {
        'model_name': model_name,
        'max_val_acc': np.amax(histories[model_name]["val_acc"]),
        'error': error,
        'f1_score': f1,
        'roc_auc': roc_auc
    }
    performance_df = performance_df.append(new_row, ignore_index=True)
    performance_df.to_csv('model_performance.csv', index=False)

In [ ]:
performance_df = open_performance()
performance_df

# Model Functions

The following functions play a part in the training of a model. 

The callbacks `ModelCheckpoint` and `EarlyStopping` are implemented during the training of the model to stop model training before overfitting.

In [ ]:
def mc(title):
    return ModelCheckpoint('./weights/{}.h5'.format(title), 
                           monitor='val_acc', mode='max', verbose=1, save_best_only=True)

es = EarlyStopping(monitor='val_acc', mode='max', min_delta=0.001, verbose=1, patience=5, restore_best_weights=True)

Since all models follow a similar baseleine, a baseline build function `build_model()` is defined with the hyperparameters optimised as default parameters to the function. A general `fit()` function is also called, which trains the model and saves the model history.

In [ ]:
def build_model(tl_model, lr=1e-4, drop_rate=0.5, stacked=False):
    model = models.Sequential()
    model.add(tl_model)      
    model.add(layers.Flatten())
    if stacked:
        model.add(layers.Dense(1024, activation='relu'))
        if drop_rate > 0:
            model.add(layers.Dropout(drop_rate))
    model.add(layers.Dense(1024, activation='relu'))
    if drop_rate > 0:
        model.add(layers.Dropout(drop_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', 
                  optimizer=SGD(lr=lr, momentum=0.9),
                  metrics=['acc'])
    
    model.summary()
    return model

In [ ]:
def fit(model, title, generator):
    train_gen = generator['train_gen']
    validation_gen = generator['validation_gen']
    
    history = model.fit(train_gen, 
                      steps_per_epoch=train_gen.samples/train_gen.batch_size, 
                      epochs=2,
                      validation_data=validation_gen,
                      validation_steps=validation_gen.samples/validation_gen.batch_size,
                      verbose=2,
                      callbacks=[es, mc(title)])
    
    histories[title] = history.history
    save_history()

The following functions are used for model evaluation:
1. `plot_curves()` plot the training and validation curves based on the model's training history. `compare_curves()` is a similar method but plots the training histories of multiple models, as defined by `model_names`, on the same plot.
2. `model_analysis()` performs a complete analysis on the performance of the model on the validation dataset. This includes computing the error_rate on the validation dataset, f1_score, roc_auc_score. For greater analysis, the confusion matrix is also plotted alongside a comprehensive classification_report which is generated and saved.
3. `compare_metrics()` visualises the performance of accuracy, f1 and auc scores of multiple models, as defined by `model_names`, on the same plot through bar graphs.

In [88]:
def plot_curves(model_name):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
    
    h = histories[model_name]

    ax1.plot(h["loss"], label="Training")
    ax1.plot(h["val_loss"], label="Validation")
    ax1.set_xlabel("# Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="best")

    ax2.plot(h["acc"], label="Training")
    ax2.plot(h["val_acc"], label="Validation")
    ax2.set_xlabel("# Epochs")
    ax2.set_ylabel("Accuracy")
    ax2.legend(loc="best")

    fig.suptitle("{} Training History".format(model_name))
    plt.savefig("results/acc_loss_plots/{}_Curves.png".format(model_name))

    plt.show()
    
    
def compare_curves(model_names, labels, title):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
    
    for i, name in enumerate(model_names):
        h = histories[name]
        ax1.plot(h["val_loss"], label=labels[i])
        ax2.plot(h["val_acc"], label=labels[i])

    ax1.set_xlabel("# Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="best")
    
    ax2.set_xlabel("# Epochs")
    ax2.set_ylabel("Accuracy")
    ax2.legend(loc="best")
    
    fig.suptitle("{} Training Comparison".format(title))
    plt.savefig("results/acc_loss_plots/{}_Comparison.png".format(title))

    plt.show()

In [84]:
def model_analysis(model, title, pred_gen):
    print('Results for {}'.format(title))
    prediction = model.predict(pred_gen,
                             steps=pred_gen.samples/pred_gen.batch_size,
                             verbose=2)
    predicted_classes = prediction.flatten()
    # Threshold output
    predicted_classes[predicted_classes>=0.5] = 1
    predicted_classes[predicted_classes<0.5] = 0

    actual = pred_gen.classes
    errors = np.where(predicted_classes != actual)[0]
    error_rate = len(errors)/pred_gen.samples
    print("Error rate {}".format(error_rate))
    
    # Geenerate confusion matrix
    genders = ['Female', 'Male']
    cm = confusion_matrix(actual, predicted_classes)
    sns.heatmap(cm, annot=True, cmap='Blues', fmt="d",
              xticklabels=genders, yticklabels=genders)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('results/cm/{}_cm.png'.format(title))
    plt.show()

    # Generate classification report
    print(classification_report(actual, predicted_classes, target_names=genders))
    clsf_report = pd.DataFrame(classification_report(actual, predicted_classes, target_names=genders, output_dict=True)).transpose()
    clsf_report.to_csv('results/clfr/{}_cr.csv'.format(title), index= True)

    # Compile all metrics and add to csv
    f1 = f1_score(actual, predicted_classes, average='weighted')
    roc_auc = roc_auc_score(actual, predicted_classes)
    save_performance(title, error_rate, f1, roc_auc)

In [89]:
def compare_metrics(model_names, labels, title):
    subset_df = performance_df.loc[performance_df['model_name'].isin(model_names)]
    subset_df.plot(x="model_name", y=["max_val_acc", "f1_score", "roc_auc"], kind="bar")
    
    val = subset_df.loc[:, ["max_val_acc", "f1_score", "roc_auc"]].values
    plt.gca().set_ylim(bottom=np.amin(val)-0.005, top=np.amax(val)+0.001)
    plt.ylabel("Metric Value")
    plt.legend(loc="best")
    
    plt.title("{} Comparison".format(title))
    plt.savefig("results/metrics/{}_Comparison.png".format(title))
    plt.show()

# Model Architectures

In this section, the various architectures are instantiated with everything except the last block of each transfer learning model is frozen using `layer.trainable = False`

## VGG16

VGG16 takes in an input image size of 224

In [ ]:
generators = generate_data()

In [ ]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_tensor=None, input_shape=(224,224,3))
vgg16_net.summary()

In [ ]:
for layer in vgg16_net.layers[:-5]:
    layer.trainable = False

In [ ]:
model = build_model(vgg16_net, lr=1e-3)
fit(model, "vgg16", generators)

In [ ]:
plot_curves("vgg16")
model_analysis(model, "vgg16", generators["predict_gen"])

## InceptionV3

InceptionV3 takes in an input image size of 224

In [ ]:
generators = generate_data()

In [ ]:
inception_net = InceptionV3(weights='imagenet', include_top=False, input_tensor=None, input_shape=(224,224,3))
inception_net.summary()

In [ ]:
for layer in inception_net.layers[:-22]:
    layer.trainable = False

In [ ]:
model = build_model(inception_net)
fit(model, "inceptionV3", generators)

In [ ]:
plot_curves("inceptionV3")
model_analysis(model, "inceptionV3", generators["predict_gen"])

## ResNet50

ResNet50 takes in an input image size of 224

In [ ]:
generators = generate_data()

In [ ]:
resnet50_conv = ResNet50(weights='imagenet', include_top=False, input_tensor=None, input_shape=(224,224,3))
resnet50_conv.summary()

In [ ]:
for layer in resnet50_conv.layers[:143]:
    layer.trainable = False

In [ ]:
model = build_model(resnet50_conv)
fit(model, "resnet50", generators)

In [ ]:
plot_curves("resnet50")
model_analysis(model, "resnet50", generators["predict_gen"])

## Xception

Xception takes in an input image size of 224

In [ ]:
generators = generate_data()

In [ ]:
xception_model = Xception(weights='imagenet', include_top=False, input_tensor=None, input_shape=(224,224,3))
xception_model.summary()

In [ ]:
for layer in xception_model.layers[:-6]:
    layer.trainable = False

In [ ]:
model = build_model(xception_model)
fit(model, "xception", generators)

In [ ]:
plot_curves("xception")
model_analysis(model, "xception", generators["predict_gen"])

## InceptionResnetV2

Xception takes in an input image size of 229

In [ ]:
generators = generate_data(img_size=229)

In [ ]:
inceptionresnetv2_conv = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=None, input_shape=(299,299,3))
inceptionresnetv2_conv.summary()

In [ ]:
for layer in inceptionresnetv2_conv.layers[:759]:
    layer.trainable = False

In [ ]:
model = build_model(inceptionresnetv2_conv)
fit(model, "IncResV2", generators)

In [ ]:
plot_curves("IncResV2")
model_analysis(model, "IncResV2", generators["predict_gen"])

## DenseNet201

Xception takes in an input image size of 224

In [ ]:
generators = generate_data()

In [ ]:
denseNet_model = DenseNet201(weights='imagenet', include_top=False, input_tensor=None, input_shape=(224,224,3))
denseNet_model.summary()

In [ ]:
for layer in denseNet_model.layers[:-9]:
    layer.trainable = False

In [ ]:
model = build_model(denseNet_model)
fit(model, "densenet", generators)

In [ ]:
plot_curves("densenet")
model_analysis(model, "densenet", generators["predict_gen"])

## Overall Comparison across Base Models

Plot the comparison curves across the 6 base models

In [ ]:
model_names = ["vgg16", "inceptionV3", "resnet50", "xception", "IncResV2", "densenet"]
compare_curves(model_names, model_names, "Base_Comparison")
compare_metrics(model_names, model_names, "Base_Comparison")

# Hyperparameter Optimisation

The 3 best models - VGG16, InceptionResnetV2 and DenseNet, are chosen and will have their parameters (batch size, dropout rate, learning rate and presence of stacked fully connected layers). Hence, the following functions were created to prevent any duplication of code

In [ ]:
OPTIMAL_ARCH = {
    "vgg16": 224,
    "IncResV2": 229,
    "densenet": 224
}

In [ ]:
def create_tlModel(arch):
    if arch == "vgg16":
        tl_model = VGG16(weights='imagenet', include_top=False, input_tensor=None, input_shape=(224,224,3))
        for layer in tl_model.layers[:-5]:
            layer.trainable = False
    elif arch == "IncResV2":
        tl_model = inceptionresnetv2_conv = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor=None, input_shape=(299,299,3))
        for layer in tl_model.layers[:759]:
            layer.trainable = False
    elif arch == "densenet":
        tl_model = DenseNet201(weights='imagenet', include_top=False, input_tensor=None, input_shape=(224,224,3))
        for layer in tl_model.layers[:-9]:
            layer.trainable = False
    else:
        print("model not found")
        tl_model = None
    return tl_model

## Optimise Batch Size

3 different batch sizes are considered - 64, 128 (base model) and 256. Therefore, each of the three architectures are then rebuilt and trained with the 2 different batch sizes - through the different image generators.

In [ ]:
BATCH_SIZES = [64, 256]
labels = ["bs-128", "bs-64", "bs-256"]

In [ ]:
for arch, img_size in OPTIMAL_ARCH.items():
    for bs in BATCH_SIZES:
        generators = generate_data(img_size=img_size, batch_size=bs)
        model_name = "{}_bs-{}".format(arch, bs)
        lr = 1e-3 if arch == "vgg16" else 1e-4
        model = build_model(create_tlModel(arch), lr=lr)
        fit(model, model_name, generators)
        model_analysis(model, model_name, generators["predict_gen"])

In [ ]:
for arch in OPTIMAL_ARCH.keys():
    model_names = [arch]
    for bs in BATCH_SIZES:
        model_names.append("{}_bs-{}".format(arch, bs))
    compare_curves(model_names, labels, "{}_bs-Comparison".format(arch))
    compare_metrics(model_names, labels, "{}_bs-Comparison".format(arch))

## Dropout Rate

3 different dropout values are considered - 0, 0.2 and 0.5 (base model). Therefore each of the three architectures are then rebuilt and trained with the 2 different dropout values - as defined by the parameter `drop_rate` in the `build_model()` function.

In [ ]:
DROP_RATES = [0, 0.2]
labels = ["dr-0.5", "dr-0", "dr-0.2"]

In [ ]:
for arch, img_size in OPTIMAL_ARCH.items():
    generators = generate_data(img_size=img_size)
    for dr in DROP_RATES:
        model_name = "{}_dr-{}".format(arch, dr)
        lr = 1e-3 if arch == "vgg16" else 1e-4
        model = build_model(create_tlModel(arch), lr=lr, drop_rate=dr)
        fit(model, model_name, generators)
        model_analysis(model, model_name, generators["predict_gen"])

In [ ]:
for arch in OPTIMAL_ARCH.keys():
    model_names = [arch]
    for dr in DROP_RATES:
        model_names.append("{}_dr-{}".format(arch, dr))
    compare_curves(model_names, labels, "{}_dr-Comparison".format(arch))
    compare_metrics(model_names, labels, "{}_dr-Comparison".format(arch))

## Optimise Learning Rate

3 different learning rates are considered - 0.01, 0.001 and 0.0001 (base model). Therefore each of the three architectures are then rebuilt and trained with the 2 different learning rates - as defined by the parameter `lr` in the `build_model()` function. The below code is slightly less straightford since the base vgg16 model differs from the rest of the model, starting with a higher learning rate. 

In [ ]:
LEARNING_RATES = [1e-2, 1e-3]
labels = ["lr-0.0001", "lr-0.01", "lr-0.001"]

In [ ]:
for arch, img_size in OPTIMAL_ARCH.items():
    generators = generate_data(img_size=img_size)
    for lr in LEARNING_RATES:
        if arch == "vgg16" and lr == 1e-3:
            model_name = "vgg16_lr-0.0001"
            model = build_model(create_tlModel(arch), lr=1e-4)
        else:
            model_name = "{}_lr-{}".format(arch, lr)
            model = build_model(create_tlModel(arch), lr=lr)
        fit(model, model_name, generators)
        model_analysis(model, model_name, generators["predict_gen"])

In [ ]:
for arch in OPTIMAL_ARCH.keys():
    model_names = [arch]
    if arch !== "vgg16":
        for lr in LEARNING_RATES:
            model_names.append("{}_lr-{}".format(arch, lr))
        compare_curves(model_names, labels, "{}_lr-Comparison".format(arch))
        compare_metrics(model_names, labels, "{}_lr-Comparison".format(arch))
    else:
        model_names.extend(["vgg16_lr-0.01", "vgg16_lr-0.0001"])
        compare_curves(model_names, ["lr-0.001", "lr-0.01", "lr-0.0001"], "vgg16_lr-Comparison")
        compare_metrics(model_names, ["lr-0.001", "lr-0.01", "lr-0.0001"], "vgg16_lr-Comparison")

## Optimise Stacking

More fully connected layers are optionally added to the base model. Each of the three architectures are then rebuilt and trained with the stacking - as defined by the parameter `stacked` in the `build_model()` function.

In [ ]:
labels = ["unstacked, stacked"]

In [ ]:
for arch, img_size in OPTIMAL_ARCH.items():
    generators = generate_data(img_size=img_size)
    model_name = "{}_stacked".format(arch)
    lr = 1e-3 if arch == "vgg16" else 1e-4
    model = build_model(create_tlModel(arch), lr=lr, stacked=True)
    fit(model, model_name, generators)
    model_analysis(model, model_name, generators["predict_gen"])

In [ ]:
for arch in OPTIMAL_ARCH.keys():
    model_names = [arch]
    model_names.append("{}_stacked".format(arch))
    compare_curves(model_names, labels, "{}_stacked-Comparison".format(arch))
    compare_metrics(model_names, labels, "{}_stacked-Comparison".format(arch))

## Combining Optimal Hyperparameters

In this stage we combine all ideal hyperparameters together to generate the optimal model for this architecture. It is then compared with its base model and each other.

The optimal parameters for each model are as follows:

|          | batch_size | learning_rate | dropout_rate | is_stacked |
|:--------:|:----------:|:-------------:|:------------:|:----------:|
| vgg16    | 64         | 1e-3          | 0.5          | True       |
| IncResV2 | 128        | 1e-2          | 0.5          | False      |
| densenet | 128        | 1e-4          | 0.5          | True       |

Since only 1 parameter has changed for both ResNet50 and DenseNet201 architectures, the respective already trained models - `IncResV2_lr-0.01` and `densenet_stacked` is used and a new model is created for vgg16.

In [ ]:
# Optimal VGG16
generators = generate_data(batch_size=64)
model = build_model(create_tlModel("vgg16"), lr=1e-3, stacked=True)
fit(model, "vgg16_optimal", generators)
model_analysis(model, "vgg16_optimal", generators["predict_gen"])

In [ ]:
model_names = ["vgg16_optimal", "IncResV2_lr-0.01", "densenet_stacked"]
compare_curves(model_names, OPTIMAL_ARCH.keys(), "optimal_Comprison")
compare_metrics(model_names, OPTIMAL_ARCH.keys(), "optimal_Comprison")

# Finetuning Models

In this section, we build open our findings of the vgg16 model to further finetune the pretrained transfer learning portion of our model to achieve incrememtal improvements with a signifantly lower learning rate (1e-5).

In [ ]:
generators = generate_data(batch_size=64)

The first experiment is to unfreeze all layers and train the model.

In [ ]:
model = models.load_model("./weights/vgg16_optimal.h5")
for layer in model.layers[0].layers:
    layer.trainable = True

print("Total trainable weights: {}".format(len(model.trainable_weights)))
model.summary()

In [ ]:
fit(model, "vgg16_unfreezeAll", generators)
model_analysis(model, "vgg16_unfreezeAll", generators["predict_gen"])

The second experiment includes freezing back the last block and unfreezing the first block of the preetrained model.

In [ ]:
model = models.load_model("./weights/vgg16_optimal.h5")
for layer in model.layers[0].layers[1:4]:
    layer.trainable = True
for layer in model.layers[0].layers[-5:]:
    layer.trainable = False

print("Total trainable weights: {}".format(len(model.trainable_weights)))
model.summary()

In [ ]:
fit(model, "vgg16_unfreezeTop", generators)
model_analysis(model, "vgg16_unfreezeTop", generators["predict_gen"])

The last experiment includes unfreezing the first block of the preetrained model with the last block still unfrozen.

In [ ]:
model = models.load_model("./weights/vgg16_optimal.h5")
for layer in model.layers[0].layers[1:4]:
    layer.trainable = True

print("Total trainable weights: {}".format(len(model.trainable_weights)))
model.summary()

In [ ]:
fit(model, "vgg16_unfreezeTopBottom", generators)
model_analysis(model, "vgg16_unfreezeTopBottom", generators["predict_gen"])

All the models are then compared with the original base models.

In [ ]:
model_names = ["vgg16_optimal", "vgg16_unfreezeAll", "vgg16_unfreezeTop", "vgg16_unfreezeTopBottom"]
compare_curves(model_names, model_names, "finetuning_Comparison")
compare_metrics(model_names, model_names, "finetuning_Comparison")

# Performance of Best Model on Test Set

This final stage evaluates the performance of the best model `vgg16_optimal` on the unseen test set. A complete model analysis is performed and finally, a few samples which the model has gotten correct and wrong for each class are plotted.

In [ ]:
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  x_col="path", y_col="gender", 
                                                  target_size=(224,224), batch_size=64, 
                                                  class_mode='binary', shuffle=False)

model = load_model("./weights/vgg16_optimal.h5")

In [ ]:
model_analysis(model, "vgg16_optimal_test")

Obtain the predictions for each sample. Since a sigmoid activation was used, thresholding is performed.

In [ ]:
prediction = model.predict(test_generator,
                            steps=test_generator.samples/test_generator.batch_size,
                            verbose=1)
predicted_classes = prediction.flatten()
predicted_classes[predicted_classes>=0.5] = 1
predicted_classes[predicted_classes<0.5] = 0

actual = test_generator.classes

Comparing the predicted classes with the actual classes, and the indice of the samples are split based on if it is correctly classified or not.

In [ ]:
errorInd = np.where(predicted_classes != actual)[0]
correctInd = np.where(predicted_classes == actual)[0]

A 4 by 5 matrix of images are plotted where the four rows represent - males classified as male, female classified as female, male classified as female, female classified as male. For each row, 5 sample images are plotted.

In [ ]:
columns, rows = 5, 4
fig, big_axes = plt.subplots( figsize=(10, 10) , nrows=rows, ncols=1, sharey=True)

row_names = ["Male Correct", "Female Correct", "Male Wrong", "Female Wrong"]
for r, big_ax in enumerate(big_axes, start=1):
    big_ax.set_title(row_names[r-1])
    # Turn off axis lines and ticks of the big subplot 
    # obs alpha is 0 in RGBA string!
    big_ax.tick_params(labelcolor=(1.,1.,1., 0.0), top='off', bottom='off', left='off', right='off')
    # removes the white frame
    big_ax._frameon = False

i = 0 # Ind to iterate through the split indexes
fem, male = 0, 0 # Counts to ensure that only 5 samples of each gender are plotted
while fem < columns or male < columns:
    if test_df.iloc[correctInd[i]]["gender"] == "male" and male < columns:
        img = mpimg.imread(test_df.iloc[correctInd[i]]["path"])
        ax = fig.add_subplot(rows,columns,male+1) # Add to the first row
        ax.imshow(img)
        ax.set_axis_off()
        male += 1
    if test_df.iloc[correctInd[i]]["gender"] == "female" and fem < columns:
        img = mpimg.imread(test_df.iloc[correctInd[i]]["path"])
        ax = fig.add_subplot(rows,columns,fem+columns+1) # Add to the second row (unraveled indexing)
        ax.imshow(img)
        ax.set_axis_off()
        fem += 1  
    i += 1

i = 0
fem, male = 0, 0 
while fem < columns or male < columns:
    if test_df.iloc[errorInd[i]]["gender"] == "male" and male < columns:
        img = mpimg.imread(test_df.iloc[errorInd[i]]["path"])
        ax = fig.add_subplot(rows,columns,male+columns*2+1) # Add to the third row
        ax.imshow(img)
        ax.set_axis_off()
        male += 1
    if test_df.iloc[errorInd[i]]["gender"] == "female" and fem < columns:
        img = mpimg.imread(test_df.iloc[errorInd[i]]["path"])
        ax = fig.add_subplot(rows,columns,fem+columns*3+1) # Add to the fourth row
        ax.imshow(img)
        ax.set_axis_off()
        fem += 1  
    i += 1

fig.tight_layout()
plt.show()